## 推荐系统
- 基于商品相似性的推荐
- 基于SVD矩阵分解的推荐

In [1]:
import pandas as pd
import numpy as np
import time
import sqlite3

data_home='./'

## 数据读取
在数据中需要：用户，歌曲，播放量

In [2]:
RawDataset=pd.read_csv(filepath_or_buffer=data_home+'train_triplets.txt',
                    sep='\t',header=None,
                   names=['user','song','play_count'])

In [3]:
RawDataset.head(10)

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBVFZR12A6D4F8AE3,1
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXALG12A8C13C108,1
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1


In [4]:
RawDataset.shape

(48373586, 3)

In [5]:
RawDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48373586 entries, 0 to 48373585
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user        object
 1   song        object
 2   play_count  int64 
dtypes: int64(1), object(2)
memory usage: 1.1+ GB


## 统计每个用户的播放总量


In [6]:
UserPlayDict={}
with open(data_home+'train_triplets.txt') as f:
    for line_number,line in enumerate(f):
        user=line.split('\t')[0] #遍历每一行的第一个元素为用户名
        play_count=int(line.split('\t')[2])
        if user in UserPlayDict:
            play_count +=UserPlayDict[user] #已存在此用户的信息时，只更新 播放次数
            UserPlayDict.update({user:play_count})
        UserPlayDict.update({user:play_count})# 如果遍历出来的用户不在字典里，则更新一组键值对
    UserPlayList=[{'user':k,'play_count':v} for k,v in UserPlayDict.items()] #把字典里的弄到数组里？
    UserPlayDf=pd.DataFrame(UserPlayList) #将数组转换成df形式
    UserPlayDfSort=UserPlayDf.sort_values(by='play_count',ascending=False) #按照播放次数的大小排序
        

In [7]:
UserPlayDfSort.to_csv(path_or_buf='UserPlayDfSort.csv',index=False) #存入本地

In [8]:
UserPlayDfSort.head()

,user,play_count
669980,093cb74eb3c517c5179ae24caf0ebec51b24d2a2,13132
402687,119b7c88d58d0c6eb051365c103da5caf817bea6,9884
964856,3fa44653315697f42410a30cb766a4eb102080bb,8210
462404,a2679496cd0af9779a92a13ff7c6af5c81ea8c7b,7015
991089,d7d2d888ae04d16e994d6964214a1de81392ee04,6494


## 统计每一首歌的播放总量

In [ ]:
# 与统计每个用户的总播放量 方法一样，直接copy
SongPlayDict={}
with open(data_home+'train_triplets.txt') as f:
    for line_number,line in enumerate(f):
        song=line.split('\t')[1] #遍历每一行的第一个元素为用户名
        play_count=int(line.split('\t')[2])
        if song in SongPlayDict:
            play_count +=SongPlayDict[song] #已存在此用户的信息时，只更新 播放次数
            SongPlayDict.update({song:play_count})
        SongPlayDict.update({song:play_count})# 如果遍历出来的用户不在字典里，则更新一组键值对
    SongPlayList=[{'song':k,'play_count':v} for k,v in SongPlayDict.items()] #把字典里的弄到数组里？
    SongPlayDf=pd.DataFrame(SongPlayList) #将数组转换成df形式
    SongPlayDfSort= SongPlayDf.sort_values(by='play_count',ascending=False) #按照播放次数的大小排序
SongPlayDfSort.to_csv(path_or_buf='SongPlayDfSort.csv',index=False) #存入本地

In [ ]:
SongPlayDfSort.head()

## 去掉惰性用户（只听一两次歌的人）
注意要分别命名每个用户和每个歌曲的总播放量，否则下面会出现混淆，得出90%的比例

In [ ]:
TotalPlayCount=sum(SongPlayDfSort.play_count)
print(  float(  UserPlayDfSort.head(n=100000).play_count.sum()  /  TotalPlayCount ) *100 )
# 取前10万个用户的总播放次数 除 歌曲的总播放次数 ->得出占比40%

In [ ]:
float(SongPlayDfSort.head(30000).play_count.sum()/TotalPlayCount)*100
#取前3万首歌的总播放量 除 歌曲总播放量 - >得出占比78%

In [ ]:
# 经过人工分析，我们可取10w个用户，三万首歌
UserPlayDfSort_10w=UserPlayDfSort.head(100000)
SongPlayDfSort_3w=SongPlayDfSort.head(30000)
User10w = list(UserPlayDfSort_10w.user)
Song3w = list(SongPlayDfSort_3w.song)

## 记住！命名注意关键词！！！ 不要叫 list，会覆盖掉内置的

In [ ]:
UserPlayDfSort_10w.shape

## 过滤掉不符合逻辑的情况
- 比如 前10w个用户中听的歌 不包含在新的歌曲列表中
- 同样 前3w首被播放的歌 不存在于新的用户列表中

In [ ]:
RawDataset.head(10)

In [ ]:
# f_user.shape
RawDataset.shape

In [ ]:
FilterDataset=RawDataset[RawDataset.user.isin(User10w)]# 去掉不在前10w用户列表中的其他数据
SecFilterDataset=FilterDataset[FilterDataset.song.isin(Song3w)]#去掉经处理过的 且不在前3w歌曲列表中的其他数据
# 保存在本地
SecFilterDataset.to_csv(path_or_buf=data_home+'User10w_Song3w.csv',index=False)

过滤后的数据

In [ ]:
SecFilterDataset.shape 

In [ ]:
SecFilterDataset.head()

## 加入音乐详细信息
- 把db数据库信息读入（注意！只需要前3w首（还是经过其他不合理筛选的））
- 转换成df格式
- 与现有数据表合并在一起

In [ ]:
conn = sqlite3.connect(data_home+'track_metadata.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
cur.fetchall()

In [ ]:
AllMsg = pd.read_sql(con=conn, sql='select * from songs')
Song3wMsg= AllMsg[AllMsg.song_id.isin(Song3w)]# 筛选出前3万歌曲的全部详细信息

In [ ]:
Song3wMsg.to_csv(path_or_buf=data_home+'Song3wMsg.csv', index=False)

In [ ]:
Song3wMsg.shape

## 我们现有的数据


In [ ]:
Song3wMsg.head(5) #经过筛选的含有详细信息的歌曲数据

### 去掉不需要的、重复的信息（列）

In [ ]:
del(Song3wMsg['track_id'])
del(Song3wMsg['artist_mbid'])
#去掉重复的行信息
Song3wMsg=Song3wMsg.drop_duplicates(['song_id'])
Song3wMsg.head()

In [ ]:
Song3wMsg.shape #经去重处理后，变成没有重复的3w首歌曲

## merge操作
- 通过观察，得到下面这个表的song_id列与原来的new_dataset表的song列是对应的
- 进行merge操作

In [ ]:
MergedAllMsg3w=pd.merge(SecFilterDataset,Song3wMsg,
                           how='left',left_on='song',right_on='song_id')
MergedAllMsg3w.info()

In [ ]:
MergedAllMsg3w.rename(columns={'play_count':'listen_count'},inplace=True)
# 把play_count改成listen_count.无关紧要的操作
MergedAllMsg3w.head()

### 删除其他无关紧要的信息（列）

In [ ]:
# 去掉不需要的指标
del(MergedAllMsg3w['song_id'])
del(MergedAllMsg3w['artist_id'])
del(MergedAllMsg3w['duration'])
del(MergedAllMsg3w['artist_familiarity'])
del(MergedAllMsg3w['artist_hotttnesss'])
del(MergedAllMsg3w['track_7digitalid'])
del(MergedAllMsg3w['shs_perf'])
del(MergedAllMsg3w['shs_work'])
MergedAllMsg3w.info()

## 可视化(●'◡'●)
- 最流行的音乐
- 最受欢迎的歌手

In [ ]:
popular_songs=MergedAllMsg3w[['title','listen_count']].groupby('title').sum().reset_index()
# 将歌曲以歌名分组，按照被播放的次数重新排序
popular_songs.head()

注意下面有个报错（上面也发生过）
- TypeError: 'list' object is not callable
- 可能是由于自己写的东西覆盖了，目前通过.tolist()方式解决

In [ ]:
top20_songs=popular_songs.sort_values('listen_count',ascending=False).head(20)
# print(top20_songs)

import matplotlib.pyplot as plt
import numpy as np

objects=(top20_songs['title']).tolist()
y_num=np.arange(len(objects))
# performance=list(top20_songs['listen_count'])
performance=top20_songs['listen_count'].tolist()

plt.bar(y_num,performance,align='center',alpha=0.5)
plt.xticks(y_num,objects,rotation='vertical')
plt.ylabel('Count')
plt.title('Top20 popular songs')

plt.show()

最受欢迎的歌手

In [ ]:
#按歌手来统计其播放总量
popular_artist = MergedAllMsg3w[['artist_name','listen_count']].groupby('artist_name').sum().reset_index()
#排序
popular_artist_top_20 = popular_artist.sort_values('listen_count', ascending=False).head(n=20)

objects = (popular_artist_top_20['artist_name']).tolist()
y_pos = np.arange(len(objects))
performance = popular_artist_top_20['listen_count'].tolist()
#绘图 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects, rotation='vertical')
plt.ylabel('Item count')
plt.title('Most popular Artists')
 
plt.show()

### 用户播放量的分布

In [ ]:
user_count_distribution = MergedAllMsg3w[['user','title']].groupby('user').count().reset_index().sort_values(
by='title',ascending = False)
user_count_distribution.title.describe()

In [ ]:
x = user_count_distribution.title
n, bins, patches = plt.hist(x, 50, facecolor='green', alpha=0.75)
plt.xlabel('Play Counts')
plt.ylabel('Num of Users')
plt.title(r'$\mathrm{Histogram\ of\ User\ Play\ Count\ Distribution}\ $')
plt.grid(True)
plt.show()

# 推荐系统

## 1、基于物品的协同过滤

In [ ]:
import Recommenders as Recommenders
from sklearn.model_selection import train_test_split

### 简单暴力，排行榜 榜单推荐

（没看）

In [ ]:
MergedAllMsg3w_set=MergedAllMsg3w
train_data,test_data=train_test_split(MergedAllMsg3w_set,test_size = 0.40, random_state=0)
train_data.head()

In [ ]:
# 把点击量当成得分值
def create_popularity_recommendation(train_data, user_id, item_id):
    #根据指定的特征来统计其播放情况，可以选择歌曲名，专辑名，歌手名
    train_data_grouped = train_data.groupby([item_id]).agg({user_id: 'count'}).reset_index()
    #为了直观展示，我们用得分来表示其结果
    train_data_grouped.rename(columns = {user_id: 'score'},inplace=True)
    
    #排行榜单需要排序
    train_data_sort = train_data_grouped.sort_values(['score', item_id], ascending = [0,1])
    
    #加入一项排行等级，表示其推荐的优先级
    train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
    #返回指定个数的推荐结果
    popularity_recommendations = train_data_sort.head(20)
    return popularity_recommendations

In [ ]:
recommendations = create_popularity_recommendation(MergedAllMsg3w_set,'user','title')

In [ ]:
recommendations

## 基于歌曲相似度的推荐
选5000的样本容量来实验

In [ ]:
# SongPlayDfSort.info()
# AllMsg.info()
MergedAllMsg3w.info()

In [ ]:
#取5000样本
Song5k=SongPlayDfSort.head(5000)

Song5kList =list(Song5k.song)# 转换成数组
MergedAllMsg5k=MergedAllMsg3w_set[MergedAllMsg3w_set.song.isin(Song5kList)]

In [ ]:
Song5k.info()

In [ ]:
MergedAllMsg5k.info()

## 计算相似度得到推荐结果

In [ ]:
import Recommenders as Recommenders
train_data, test_data = train_test_split(MergedAllMsg5k, test_size = 0.30, random_state=0)

#调用模型
is_model = Recommenders.item_similarity_recommender_py()

is_model.create(train_data, 'user', 'title')

user_id =list(train_data.user)[7] #注意先转换成数组，再索引  !!! 
user_items = is_model.get_user_items(user_id)

In [ ]:
'''
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id
        
    def get_user_items(self, user):
        # 拿到当前用户听过的所有歌（一首歌可能出现很多次）
        user_data = self.train_data[self.train_data[self.user_id] == user]
        # 变成数组+去重
        user_items = list(user_data[self.item_id].unique())
        return user_items
""

In [ ]:
user_id

In [ ]:
len(user_items)

In [ ]:
#执行推荐
is_model.recommend(user_id)

In [ ]:
''''
    def recommend(self, user):

        #1、得到用户听过的所有歌
        user_songs = self.get_user_items(user)    
            
        print("No. of unique songs for the user: %d" % len(user_songs))
        
        #2、得到数据集中所有的歌
        all_songs = self.get_all_items_train_data()
        
        print("no. of unique songs in the training set: %d" % len(all_songs))
         
        #3、构建矩阵
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        
        #4、用交并集计算
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
                
        return df_recommendations

## 基于矩阵分解（SVD）的推荐

相似度计算的方法看起来比较简单就是实现出来，但是当数据较大的时候计算的时间消耗实在太大了，对每一个用户都需要多次遍历整个数据集来进行计算，矩阵分解的方法是当下更常使用的方法。

奇异值分解(Singular Value Decomposition，SVD)是矩阵分解中一个经典方法
- 奇异值分解的基本出发点跟隐语义模型有些类似都是将大矩阵转换成小矩阵的组合
- 在SVD中我们所需的数据是用户对商品的打分，
- 但是我们现在的数据集中只有用户播放歌曲的情况并没有实际的打分值
- 所以我们还得自己来定义一下用户对每个歌曲的评分值。
如果一个用户喜欢某个歌曲，应该经常播放这个歌曲，相反如果不喜欢某个歌曲，那播放次数肯定就比较少了。
#### 用户对歌曲的打分值，定义为：用户播放该歌曲数量/该用户播放总量。

#### 1、统计listen_count

In [ ]:
MergedAllMsg5k.info()

In [ ]:
# 计算歌曲被用户播放的总量
Msg5kCountSum=MergedAllMsg5k[['user','listen_count']].groupby('user').sum().reset_index()
Msg5kCountSum.info()

In [ ]:
# 测试玩的
# 选择其中的两列以song分组，计算总和？..
# Msg5kCountSum_song=MergedAllMsg5k[['song','listen_count']].groupby('song').sum().reset_index()
# Msg5kCountSum_song.info()

In [ ]:
Msg5kCountSum.rename(columns={'listen_count':'total_count'},inplace=True)
Msg5kCountSum.head()

In [ ]:
#Merge操作
MergedMsg5kCountSum=pd.merge(MergedAllMsg5k,Msg5kCountSum)
MergedMsg5kCountSum.head()

In [ ]:
MergedMsg5kCountSum.info()

#### 2、计算比值

In [ ]:
MergedMsg5kCountSum['fractional_play_count'] = MergedMsg5kCountSum['listen_count']/MergedMsg5kCountSum['total_count']
MergedMsg5kCountSum.head()

In [ ]:
# 例子
# 某用户听不同歌的次数占总次数的比率
MergedMsg5kCountSum[MergedMsg5kCountSum.user =='d6589314c0a9bcbca4fee0c93b14bc402363afea'][['user','song','listen_count','fractional_play_count']].head()

#### 3、引入矩阵分解

In [ ]:
from scipy.sparse import coo_matrix

small_set = MergedMsg5kCountSum
#去重
user_codes = small_set.user.drop_duplicates().reset_index()
song_codes = small_set.song.drop_duplicates().reset_index()
#改名
user_codes.rename(columns={'index':'user_index'}, inplace=True) 
song_codes.rename(columns={'index':'song_index'}, inplace=True)
# 把对应的索引值作为内容？
user_codes['us_index_value'] = list(user_codes.index)
song_codes['so_index_value'] = list(song_codes.index)

small_set = pd.merge(small_set,song_codes,how='left')
small_set = pd.merge(small_set,user_codes,how='left')
#映射
mat_set= small_set[['us_index_value','so_index_value','fractional_play_count']]

# 得到全部内容
data_array = mat_set.fractional_play_count.values
row_array = mat_set.us_index_value.values
col_array = mat_set.so_index_value.values

# 重新构造一个矩阵
data_sparse = coo_matrix((data_array, (row_array, col_array)),dtype=float)

In [ ]:
user_codes

In [ ]:
mat_set

In [ ]:
row_array

In [ ]:
 data_sparse #稀疏矩阵 （  0比较多的矩阵  ）

### 使用SVD方法来进行矩阵分解

矩阵构造好了之后我们就要执行SVD矩阵分解了，这里还需要一些额外的工具包来帮助我们完成计算，scipy就是其中一个好帮手了，里面已经封装好了SVD计算方法。

In [ ]:
import math as mt
from scipy.sparse.linalg import * #used for matrix multiplication
from scipy.sparse.linalg import svds
from scipy.sparse import csc_matrix

In [ ]:
def compute_svd(urm, K):
    U, s, Vt = svds(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csc_matrix(U, dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

def compute_estimated_matrix(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 
    print('rightTerm-shape',rightTerm.shape)
    max_recommendation = 250
    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    recomendRatings = np.zeros(shape=(MAX_UID,max_recommendation ), dtype=np.float16)
    for userTest in uTest:
        print('U[userTest,:]',U[userTest,:].shape)
        prod = U[userTest, :]*rightTerm
        estimatedRatings[userTest, :] = prod.todense()
        recomendRatings[userTest, :] = (-estimatedRatings[userTest, :]).argsort()[:max_recommendation]
    return recomendRatings

In [ ]:
K=50 # 选择50个特征值
urm = data_sparse 
MAX_PID = urm.shape[1]
MAX_UID = urm.shape[0]

U, S, Vt = compute_svd(urm, K)

In [ ]:
# 选这几个索引的用户进行推荐
uTest = [4,5,6,7,8,873,23]

uTest_recommended_items = compute_estimated_matrix(urm, U, S, Vt, uTest, K, True)

In [ ]:
for user in uTest:
    print("Recommendation for user with user id {}". format(user))
    rank_value = 1
    for i in uTest_recommended_items[user,0:10]:
        song_details = small_set[small_set.so_index_value == i].drop_duplicates('so_index_value')[['title','artist_name']]
        print("The number {} recommended song is {} BY {}".format(rank_value, list(song_details['title'])[0],list(song_details['artist_name'])[0]))
        rank_value+=1

这里对每一个用户都得到了其对应的推荐结果，并且将结果按照得分值进行排序。


本章我们选择了音乐数据集来进行个性化推荐任务，首先对数据进行预处理和整合，选择两种方法分别完成推荐任务。在相似度计算中根据用户所听过的歌曲在候选集中选择与其最相似的歌曲，存在的问题就是计算时间消耗太多，每一个用户都需要重新计算一遍才能得出推荐结果。在SVD矩阵分解的方法中，我们首先构建评分矩阵，对其进行SVD分解，然后选择待推荐用户，还原得到其对所有歌曲的估测评分值，最后排序返回结果即可。

In [ ]:
uTest = [27513]
#Get estimated rating for test user
print("Predictied ratings:")
uTest_recommended_items = compute_estimated_matrix(urm, U, S, Vt, uTest, K, True)

In [ ]:
for user in uTest:
    print("Recommendation for user with user id {}". format(user))
    rank_value = 1
    for i in uTest_recommended_items[user,0:10]:
        song_details = small_set[small_set.so_index_value == i].drop_duplicates('so_index_value')[['title','artist_name']]
        print("The number {} recommended song is {} BY {}".format(rank_value, list(song_details['title'])[0],list(song_details['artist_name'])[0]))
        rank_value+=1